In [1]:
!pip install PyMuPDF Pillow

  Using cached pymupdf-1.26.4-cp39-abi3-macosx_11_0_arm64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.4-cp39-abi3-macosx_11_0_arm64.whl (22.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [Pillow]2m1/2 [Pillow]


In [3]:

"""
Extracteur de cartes Victory & Valour (Marges manuelles)
========================================================

Version simple du code original avec possibilité de configurer manuellement
les marges en haut et en bas de chaque page.

Dépendances requises:
    pip install PyMuPDF Pillow

Usage:
    python extracteur_cartes_marges_manuelles.py
"""

import os
import shutil
import fitz  # PyMuPDF
from PIL import Image

# ===============================================
# CONFIGURATION DES MARGES (À AJUSTER MANUELLEMENT)
# ===============================================

# Marges en pixels - ajustez ces valeurs selon vos besoins
MARGE_HAUT = 155      # Pixels à ignorer en haut de la page
MARGE_BAS = 155       # Pixels à ignorer en bas de la page
MARGE_GAUCHE =130     # Pixels à ignorer à gauche de la page  
MARGE_DROITE = 120     # Pixels à ignorer à droite de la page

# ===============================================

def extract_cards_with_manual_margins(pdf_path, output_folder="cartes_individuelles"):
    """
    Extrait 54 cartes individuelles d'un PDF en utilisant des marges configurables manuellement.

    Args:
        pdf_path (str): Chemin vers le fichier PDF
        output_folder (str): Dossier de sortie pour les cartes individuelles

    Returns:
        bool: True si l'extraction s'est bien passée, False sinon
    """

    # Créer le dossier de sortie s'il n'existe pas
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder)

    try:
        # Ouvrir le PDF avec PyMuPDF
        pdf_document = fitz.open(pdf_path)
        print(f"PDF ouvert : {len(pdf_document)} pages")

        if len(pdf_document) != 6:
            print(f"Attention : Le PDF contient {len(pdf_document)} pages au lieu de 6 attendues")

    except Exception as e:
        print(f"Erreur lors de l'ouverture du PDF : {e}")
        return False

    card_number = 1

    print(f"\nMarges configurées :")
    print(f"  Haut: {MARGE_HAUT} pixels")
    print(f"  Bas: {MARGE_BAS} pixels")
    print(f"  Gauche: {MARGE_GAUCHE} pixels")
    print(f"  Droite: {MARGE_DROITE} pixels")

    for page_num in range(min(6, len(pdf_document))):  # Traiter au maximum 6 pages
        print(f"\nTraitement de la page {page_num + 1}...")

        # Obtenir la page
        page = pdf_document[page_num]

        # Convertir la page en image (haute résolution)
        mat = fitz.Matrix(300/72, 300/72)  # 300 DPI
        pix = page.get_pixmap(matrix=mat)

        # Convertir en image PIL
        img_data = pix.tobytes("png")
        page_image = Image.open(fitz.io.BytesIO(img_data))

        # Dimensions de la page
        page_width, page_height = page_image.size
        print(f"  Dimensions de la page: {page_width}x{page_height}")

        # Calculer la zone utile (sans les marges)
        zone_gauche = MARGE_GAUCHE
        zone_droite = page_width - MARGE_DROITE
        zone_haut = MARGE_HAUT
        zone_bas = page_height - MARGE_BAS

        zone_largeur = zone_droite - zone_gauche
        zone_hauteur = zone_bas - zone_haut

        print(f"  Zone de découpage: {zone_largeur}x{zone_hauteur}")
        print(f"  De ({zone_gauche},{zone_haut}) à ({zone_droite},{zone_bas})")

        # Calculer les dimensions de chaque carte (3x3 grille dans la zone utile)
        largeur_carte = zone_largeur // 3
        hauteur_carte = zone_hauteur // 3

        print(f"  Taille de chaque carte: {largeur_carte}x{hauteur_carte}")

        # Extraire chaque carte de la grille 3x3
        for rangee in range(3):
            for colonne in range(3):
                # Calculer les coordonnées de découpe
                gauche = zone_gauche + colonne * largeur_carte
                haut = zone_haut + rangee * hauteur_carte
                droite = gauche + largeur_carte
                bas = haut + hauteur_carte

                print(f"    Carte {card_number}: ({gauche},{haut}) -> ({droite},{bas})")

                # Extraire la carte
                carte = page_image.crop((gauche, haut, droite, bas))

                # Nom du fichier de sortie
                nom_fichier = f"{card_number}.png"
                chemin_sortie = os.path.join(output_folder, nom_fichier)

                # Sauvegarder la carte en PNG
                carte.save(chemin_sortie, "PNG", optimize=True)

                print(f"      ✅ Sauvegardée : {nom_fichier}")
                card_number += 1

                # Arrêter après 54 cartes
                if card_number > 54:
                    break

            if card_number > 54:
                break

        if card_number > 54:
            break

    # Fermer le PDF
    pdf_document.close()

    total_cartes = card_number - 1
    print(f"\n=== TERMINÉ ===")
    print(f"📊 {total_cartes} cartes extraites dans le dossier '{output_folder}'")

    # Vérifier que nous avons bien 54 cartes
    if total_cartes == 54:
        print("✅ Toutes les 54 cartes ont été extraites avec succès !")
    else:
        print(f"⚠️  Attention : {total_cartes} cartes extraites au lieu de 54")

    return True

def main():
    """
    Fonction principale pour traiter le PDF Victory Valour
    """
    pdf_file = "Printable-A4-Victory-Valour.pdf"

    print("=" * 60)
    print("  EXTRACTEUR DE CARTES VICTORY & VALOUR")
    print("  (Version avec marges manuelles)")
    print("=" * 60)
    print()
    print("📝 Pour ajuster les marges, modifiez les valeurs en haut du script :")
    print(f"   MARGE_HAUT = {MARGE_HAUT}")
    print(f"   MARGE_BAS = {MARGE_BAS}")
    print(f"   MARGE_GAUCHE = {MARGE_GAUCHE}")
    print(f"   MARGE_DROITE = {MARGE_DROITE}")
    print()

    # Vérifier que le fichier existe
    if not os.path.exists(pdf_file):
        print(f"❌ Erreur : Le fichier {pdf_file} n'existe pas dans le répertoire courant.")
        print("   Assurez-vous que le PDF est dans le même dossier que ce script.")
        return

    print(f"📄 Fichier PDF trouvé : {pdf_file}")
    print("🚀 Début de l'extraction...")

    # Extraire les cartes
    success = extract_cards_with_manual_margins(pdf_file)

    if success:
        print("\n🎉 Extraction terminée avec succès !")
        print("📁 Consultez le dossier 'cartes_individuelles' pour voir toutes les cartes.")
        print("\nLes cartes sont nommées de 1.png à 54.png")
        print()
        print("💡 Si les cartes ne sont pas bien découpées :")
        print("   - Augmentez MARGE_HAUT/MARGE_BAS si il y a des traits en haut/bas")
        print("   - Augmentez MARGE_GAUCHE/MARGE_DROITE si il y a des traits sur les côtés")
        print("   - Diminuez les marges si les cartes sont trop petites")
    else:
        print("\n❌ Erreur lors de l'extraction.")
        print("Vérifiez que les dépendances sont installées :")
        print("  pip install PyMuPDF Pillow")

if __name__ == "__main__":
    main()


  EXTRACTEUR DE CARTES VICTORY & VALOUR
  (Version avec marges manuelles)

📝 Pour ajuster les marges, modifiez les valeurs en haut du script :
   MARGE_HAUT = 155
   MARGE_BAS = 155
   MARGE_GAUCHE = 130
   MARGE_DROITE = 120

📄 Fichier PDF trouvé : Printable-A4-Victory-Valour.pdf
🚀 Début de l'extraction...
PDF ouvert : 6 pages

Marges configurées :
  Haut: 155 pixels
  Bas: 155 pixels
  Gauche: 130 pixels
  Droite: 120 pixels

Traitement de la page 1...
  Dimensions de la page: 2481x3508
  Zone de découpage: 2231x3198
  De (130,155) à (2361,3353)
  Taille de chaque carte: 743x1066
    Carte 1: (130,155) -> (873,1221)
      ✅ Sauvegardée : 1.png
    Carte 2: (873,155) -> (1616,1221)
      ✅ Sauvegardée : 2.png
    Carte 3: (1616,155) -> (2359,1221)
      ✅ Sauvegardée : 3.png
    Carte 4: (130,1221) -> (873,2287)
      ✅ Sauvegardée : 4.png
    Carte 5: (873,1221) -> (1616,2287)
      ✅ Sauvegardée : 5.png
    Carte 6: (1616,1221) -> (2359,2287)
      ✅ Sauvegardée : 6.png
    Carte 7: